In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../')))
sys.path.append(os.path.abspath(os.path.join('../MONAI')))

import numpy as np
import json

import torch
import torch.nn.functional as F

import torch.distributed as dist
import torchio as tio
import gc

from monai.utils import set_determinism, first

from tqdm import tqdm
import wandb

from script.utils import *
from script.configure.LDMconfig import get_run_parser

import warnings
warnings.filterwarnings("ignore")
from data.LongDataset import LongitudinalDataset

In [ ]:
VQVAEPATH = '/NFS/FutureBrainGen/ckpt/VQGAN/dry-night-3/best_vqvae_model_dim16_reconloss0.007_ep500.pth'

In [ ]:
device = torch.device('cuda:1')
EDmodel = load_VQVAE(device, VQVAEPATH, wrap_ddp=False, local_rank=None)

In [ ]:
from types import SimpleNamespace

config = {'data_path':'/NFS/FutureBrainGen/data/long/',
          'crop_size': (96, 112, 96),         
}
config = SimpleNamespace(**config)

In [ ]:
ValDataset = LongitudinalDataset(config, _type='val', Transform=None)
val_loader = DataLoader(
    ValDataset,
    batch_size=4,
    num_workers=4,
    pin_memory=True,
    drop_last=True,
)

first_batch = first(val_loader)

In [ ]:
with torch.no_grad():
    z = EDmodel.encode_stage_2_inputs(first_batch['base_img'].to(device))

In [ ]:
z.shape

In [ ]:
first_batch['Age_B']

In [ ]:
flatten = torch.rand([4, 1, 258048])
Latent = torch.rand([4, 16, 24, 28, 24])
age_B = torch.tensor([40,50,60,70])

In [ ]:
age_B_f = torch.tensor([40, 50, 60, 70], dtype=torch.float32).view(4, 1, 1)

In [ ]:
flatten + age_B_f

In [ ]:
torch.sum(flatten, dim=(0)).shape